In [24]:
using Pkg
pkg"activate .; instantiate; precompile"

  Updating registry at `~/.julia/registries/General`
  Updating git-repo `https://github.com/JuliaRegistries/General.git`
Precompiling project...


In [4]:
using PerlaTonettiWaugh, Plots, BenchmarkTools, CSV

# Understanding why stuff is "big"

Here is the approach. I will walk through several different cases. The idea is to vary mu, varepsilon, sigma and delta to see what is going on. In each of the cases, these parameters will be fixed, the remaining four parameters (trade cost, theta, kappa, chi) are REcalibrated to match the four (updated) original moments that we used: trade, growth, mass of exporters, relative size.

## The Basline

Just to see where our current calibration is at...

In [23]:
parameters = parameter_defaults()
parameters = merge(parameters, (ρ = 0.0195, d = 5.0672, θ = 3.2941,  κ = 0.1152, χ = 1/4.7847, υ = 0.0819, μ = -0.0289, σ = 2.3664,))
settings = settings_defaults()
d_0 = parameters.d # Here is the 10 percent tariff increase
d_T = 1 + (parameters.d-1)*0.90

#d_autarky = 1 + (parameters.d-1)*2.5 

params_0 = merge(parameters, (d = d_0, )) # parameters to be used at t = 0
params_T = merge(parameters, (d = d_T, )) # parameters to be used at t = T
#params_autarky = merge(parameters, (d = d_autarky, )) # parameters to be used in autarky calc

# initial value for numerical solver on (g, z_hat, Omega)
initial_x = [0.02; 2; .57]

# solve for stationary solution at t = 0
stationary_sol_0 = stationary_algebraic(params_0, settings) # solution at t = 0
stationary_sol = stationary_algebraic(params_T, settings) # solution at t = T
#stationary_autarky = stationary_algebraic(params_autarky, initial_x) # solution at t = T

lambda_ss = 100*(consumption_equivalent(stationary_sol.U_bar, stationary_sol_0.U_bar, parameters)-1)

print("SS to SS welfare gain: ", lambda_ss,"\n")
print("Change in Trade: ", (1-stationary_sol.λ_ii) - (1-stationary_sol_0.λ_ii),"\n")
print("Growth Rates across SS: ", 100*stationary_sol.g,"   " ,100*stationary_sol_0.g, "\n")

AssertionError: AssertionError: z_hat > 1 && (Ω > 0 && g > 0)

## Case \#1: Turn off drift, mu = 0

Below are the resulting parameter values in the following order: Trade cost, theta, kappa, and chi

``5.0623    3.2944    0.0633    3.3966``

Note how the trade cost and theta are unchanged, kappa is lower, chi is lower. Chi makes sense in that the negative drift mechanichally forced people up to the threshold (just like the variance), so this is a mechanichal force to increase growth. Thus, with negative drift, to mute or tame growth, we need a larger cost of adoption. In constrast, when this is turned off we need a lower cost.

In [6]:
parameters = parameter_defaults()
parameters = merge(parameters, (ρ = 0.0195, d = 5.0623, θ = 3.2944,  κ = 0.0633, χ = 1/3.3966, υ = 0.0819, μ = 0.0, σ = 2.3664,))

d_0 = parameters.d # Here is the 10 percent tariff increase
d_T = 1 + (parameters.d-1)*0.90

#d_autarky = 1 + (parameters.d-1)*2.5 

params_0 = merge(parameters, (d = d_0, )) # parameters to be used at t = 0
params_T = merge(parameters, (d = d_T, )) # parameters to be used at t = T
#params_autarky = merge(parameters, (d = d_autarky, )) # parameters to be used in autarky calc

# initial value for numerical solver on (g, z_hat, Omega)
initial_x = [0.02; 2; .57]

# solve for stationary solution at t = 0
stationary_sol_0 = stationary_algebraic(params_0, initial_x) # solution at t = 0
stationary_sol = stationary_algebraic(params_T, initial_x) # solution at t = T
#stationary_autarky = stationary_algebraic(params_autarky, initial_x) # solution at t = T

lambda_ss = 100*(consumption_equivalent(stationary_sol.U_bar, stationary_sol_0.U_bar, parameters)-1)

print("SS to SS welfare gain: ", lambda_ss,"\n")
print("Change in Trade: ", (1-stationary_sol.λ_ii) - (1-stationary_sol_0.λ_ii),"\n")
print("Growth Rates across SS: ", 100*stationary_sol.g,"   " ,100*stationary_sol_0.g, "\n")

ErrorException: type Array has no field stationary_x0

### Discussion...

- First, welfare gains are smaller, growth goes up by a bit less. Not dramatic.
- Second, note that I had to comment out the autarky calculation. In the autarky setting it says that parameter restrictcions were violated. This was the orginal reason we let the negative drift into the calibration to give more "scope for the thing to work"

---

## Case \#2: Turn off drift, mu = 0, lower variance

As a first step, lets just do three things 0.06, 0.03, 0.005

**Summary** Several things happen from this excercise

1. **Welfare and growth systematically fall as GBM is reduced.** So the final calibration delivers a welfare gain of 4.6 percent. (1/3) of our baseline value. Much of this is because growth is not changing as much now aoubt 10 basis points. 

2. The only parameter changing across calibrations is Chi. Surprisingly, the trade costs, theta, and exporter fixed costs are virtually unchanged through out the excercise. This suggests strongly about how trade and the exporter facts basically pin down d, theta, kappa. Then the growth rate is all about chi.

3. Change in trade. This is not a mechanichal function of theta and the counterfactuall. It is changing a bit, but not always 3.29. I think this is because the trade elasticity is always a PE elasticit, hold prices fixed, change tau by a small amount, the trade changes by theta. We are not in this world. 

Using logic from the non-GBM case, the issue seems to be about how Chi is feeding into the elasticity of growth. What we are finding is that as we decrease GBM variance, that Chi is increasing (about 0.20 in baseline, 0.50 in low GBM situation), this means that the adoption cost is ???

**Below are the details and the code**

---

- The ``0.06`` calibration: ``5.0684    3.2941    0.0636    3.1965``

In [7]:
parameters = parameter_defaults()
parameters = merge(parameters, (ρ = 0.0195, d = 5.0684, θ = 3.2941,  κ = 0.0636, χ = 1/3.1965, υ = 0.06, μ = 0.0, σ = 2.3664,))

d_0 = parameters.d # Here is the 10 percent tariff increase
d_T = 1 + (parameters.d-1)*0.90

#d_autarky = 1 + (parameters.d-1)*2.5 

params_0 = merge(parameters, (d = d_0, )) # parameters to be used at t = 0
params_T = merge(parameters, (d = d_T, )) # parameters to be used at t = T
#params_autarky = merge(parameters, (d = d_autarky, )) # parameters to be used in autarky calc

# initial value for numerical solver on (g, z_hat, Omega)
initial_x = [0.02; 2; .57]

# solve for stationary solution at t = 0
stationary_sol_0 = stationary_algebraic(params_0, initial_x) # solution at t = 0
stationary_sol = stationary_algebraic(params_T, initial_x) # solution at t = T
#stationary_autarky = stationary_algebraic(params_autarky, initial_x) # solution at t = T

lambda_ss = 100*(consumption_equivalent(stationary_sol.U_bar, stationary_sol_0.U_bar, parameters)-1)

print("SS to SS welfare gain: ", lambda_ss,"\n")
print("Change in Trade: ", (1-stationary_sol.λ_ii) - (1-stationary_sol_0.λ_ii),"\n")
print("Growth Rates across SS: ", 100*stationary_sol.g,"   " ,100*stationary_sol_0.g, "\n")

ErrorException: type Array has no field stationary_x0

---

- The ``0.03`` calibration: ``5.0679    3.2943    0.0624    2.9368``

In [8]:
parameters = merge(parameters, (,))
d_0 = parameters.d # Here is the 10 percent tariff increase
d_T = 1 + (parameters.d-1)*0.90

#d_autarky = 1 + (parameters.d-1)*2.5 

params_0 = merge(parameters, (d = d_0, )) # parameters to be used at t = 0
params_T = merge(parameters, (d = d_T, )) # parameters to be used at t = T
#params_autarky = merge(parameters, (d = d_autarky, )) # parameters to be used in autarky calc

# initial value for numerical solver on (g, z_hat, Omega)
initial_x = [0.02; 2; .57]

# solve for stationary solution at t = 0
stationary_sol_0 = stationary_algebraic(params_0, initial_x) # solution at t = 0
stationary_sol = stationary_algebraic(params_T, initial_x) # solution at t = T
#stationary_autarky = stationary_algebraic(params_autarky, initial_x) # solution at t = T

lambda_ss = 100*(consumption_equivalent(stationary_sol.U_bar, stationary_sol_0.U_bar, parameters)-1)

print("SS to SS welfare gain: ", lambda_ss,"\n")
print("Change in Trade: ", (1-stationary_sol.λ_ii) - (1-stationary_sol_0.λ_ii),"\n")
print("Growth Rates across SS: ", 100*stationary_sol.g,"   " ,100*stationary_sol_0.g, "\n")

LoadError: syntax: unexpected ","

---

- The ``0.005`` calibration: ``5.0678    3.2943    0.0610    2.7949``

In [9]:
parameters = merge(parameters,,))

d_0 = parameters.d # Here is the 10 percent tariff increase
d_T = 1 + (parameters.d-1)*0.90

#d_autarky = 1 + (parameters.d-1)*2.5 

params_0 = merge(parameters, (d = d_0, )) # parameters to be used at t = 0
params_T = merge(parameters, (d = d_T, )) # parameters to be used at t = T
#params_autarky = merge(parameters, (d = d_autarky, )) # parameters to be used in autarky calc

# initial value for numerical solver on (g, z_hat, Omega)
initial_x = [0.02; 2; .57]

# solve for stationary solution at t = 0
stationary_sol_0 = stationary_algebraic(params_0, initial_x) # solution at t = 0
stationary_sol = stationary_algebraic(params_T, initial_x) # solution at t = T
#stationary_autarky = stationary_algebraic(params_autarky, initial_x) # solution at t = T

lambda_ss = 100*(consumption_equivalent(stationary_sol.U_bar, stationary_sol_0.U_bar, parameters)-1)

print("SS to SS welfare gain: ", lambda_ss,"\n")
print("Change in Trade: ", (1-stationary_sol.λ_ii) - (1-stationary_sol_0.λ_ii),"\n")
print("Growth Rates across SS: ", 100*stationary_sol.g,"   " ,100*stationary_sol_0.g, "\n")

LoadError: syntax: unexpected ","

---

## Case \#3 Vary Sigma, hold fixed GBM and drift

We will do this for a plausible range of sigma's: 2, 3, 4, 5.

**Summary** Several things happen from this excercise

1. **Welfare and growth systematically rise as Sigma increases.** So the first calibration is simmilar to what we have in the baseline, but as sigma increases, the welfare gains and the change in the growth rate increase. 

2. The interpertation here is more complicated for several reasons. Now, all parameters are changing: theta, trade costs, etc. Also, the change in the volume of trade is chaniging as well. So to interpert the elasticity of a change in growth to a change in trade is harder to see as well. With that said, this is consistent with interpertation that sigma is influanceing the infered chi paramter and in turn how responsive economic growth is to a change in trade costs.

3. This is also consistent with the experince of previous calibrations. We had a calibration with a fixed sigma of 3, but when we calibrated sigma, it pushed up to 2 something which is smaller giving a bit more modest gains....

**Below are the details and the code**

---

- Sigma = 2, calibration is ``7.5097    2.4071    0.1165    4.0249``

In [10]:
parameters = parameter_defaults()
parameters = merge(parameters, ,))

d_0 = parameters.d # Here is the 10 percent tariff increase
d_T = 1 + (parameters.d-1)*0.90

#d_autarky = 1 + (parameters.d-1)*2.5 

params_0 = merge(parameters, (d = d_0, )) # parameters to be used at t = 0
params_T = merge(parameters, (d = d_T, )) # parameters to be used at t = T
#params_autarky = merge(parameters, (d = d_autarky, )) # parameters to be used in autarky calc

# initial value for numerical solver on (g, z_hat, Omega)
initial_x = [0.02; 2; .57]

# solve for stationary solution at t = 0
stationary_sol_0 = stationary_algebraic(params_0, initial_x) # solution at t = 0
stationary_sol = stationary_algebraic(params_T, initial_x) # solution at t = T
#stationary_autarky = stationary_algebraic(params_autarky, initial_x) # solution at t = T

lambda_ss = 100*(consumption_equivalent(stationary_sol.U_bar, stationary_sol_0.U_bar, parameters)-1)

print("SS to SS welfare gain: ", lambda_ss,"\n")
print("Change in Trade: ", (1-stationary_sol.λ_ii) - (1-stationary_sol_0.λ_ii),"\n")
print("Growth Rates across SS: ", 100*stationary_sol.g,"   " ,100*stationary_sol_0.g, "\n")

LoadError: syntax: unexpected ","

- Sigma = 3, calibration is ``3.0307    4.8219    0.1417    5.9260``

In [11]:
parameters = merge(parameters, ,))

d_0 = parameters.d # Here is the 10 percent tariff increase
d_T = 1 + (parameters.d-1)*0.90

#d_autarky = 1 + (parameters.d-1)*2.5 

params_0 = merge(parameters, (d = d_0, )) # parameters to be used at t = 0
params_T = merge(parameters, (d = d_T, )) # parameters to be used at t = T
#params_autarky = merge(parameters, (d = d_autarky, )) # parameters to be used in autarky calc

# initial value for numerical solver on (g, z_hat, Omega)
initial_x = [0.02; 2; .57]

# solve for stationary solution at t = 0
stationary_sol_0 = stationary_algebraic(params_0, initial_x) # solution at t = 0
stationary_sol = stationary_algebraic(params_T, initial_x) # solution at t = T
#stationary_autarky = stationary_algebraic(params_autarky, initial_x) # solution at t = T

lambda_ss = 100*(consumption_equivalent(stationary_sol.U_bar, stationary_sol_0.U_bar, parameters)-1)

print("SS to SS welfare gain: ", lambda_ss,"\n")
print("Change in Trade: ", (1-stationary_sol.λ_ii) - (1-stationary_sol_0.λ_ii),"\n")
print("Growth Rates across SS: ", 100*stationary_sol.g,"   " ,100*stationary_sol_0.g, "\n")

LoadError: syntax: unexpected ","

- Sigma = 4, calibration is ``2.0942    7.2329    0.1748    7.6963``

In [12]:
parameters = merge(parameters, (,))

d_0 = parameters.d # Here is the 10 percent tariff increase
d_T = 1 + (parameters.d-1)*0.90

#d_autarky = 1 + (parameters.d-1)*2.5 

params_0 = merge(parameters, (d = d_0, )) # parameters to be used at t = 0
params_T = merge(parameters, (d = d_T, )) # parameters to be used at t = T
#params_autarky = merge(parameters, (d = d_autarky, )) # parameters to be used in autarky calc

# initial value for numerical solver on (g, z_hat, Omega)
initial_x = [0.02; 2; .57]

# solve for stationary solution at t = 0
stationary_sol_0 = stationary_algebraic(params_0, initial_x) # solution at t = 0
stationary_sol = stationary_algebraic(params_T, initial_x) # solution at t = T
#stationary_autarky = stationary_algebraic(params_autarky, initial_x) # solution at t = T

lambda_ss = 100*(consumption_equivalent(stationary_sol.U_bar, stationary_sol_0.U_bar, parameters)-1)

print("SS to SS welfare gain: ", lambda_ss,"\n")
print("Change in Trade: ", (1-stationary_sol.λ_ii) - (1-stationary_sol_0.λ_ii),"\n")
print("Growth Rates across SS: ", 100*stationary_sol.g,"   " ,100*stationary_sol_0.g, "\n")

LoadError: syntax: unexpected ","

- Sigma = 5, calibration is ``1.7410    9.6431    0.1977    9.4262''

In [13]:
parameters = merge(parameters, (,))

d_0 = parameters.d # Here is the 10 percent tariff increase
d_T = 1 + (parameters.d-1)*0.90

#d_autarky = 1 + (parameters.d-1)*2.5 

params_0 = merge(parameters, (d = d_0, )) # parameters to be used at t = 0
params_T = merge(parameters, (d = d_T, )) # parameters to be used at t = T
#params_autarky = merge(parameters, (d = d_autarky, )) # parameters to be used in autarky calc

# initial value for numerical solver on (g, z_hat, Omega)
initial_x = [0.02; 2; .57]

# solve for stationary solution at t = 0
stationary_sol_0 = stationary_algebraic(params_0, initial_x) # solution at t = 0
stationary_sol = stationary_algebraic(params_T, initial_x) # solution at t = T
#stationary_autarky = stationary_algebraic(params_autarky, initial_x) # solution at t = T

lambda_ss = 100*(consumption_equivalent(stationary_sol.U_bar, stationary_sol_0.U_bar, parameters)-1)

print("SS to SS welfare gain: ", lambda_ss,"\n")
print("Change in Trade: ", (1-stationary_sol.λ_ii) - (1-stationary_sol_0.λ_ii),"\n")
print("Growth Rates across SS: ", 100*stationary_sol.g,"   " ,100*stationary_sol_0.g, "\n")

LoadError: syntax: unexpected ","

## Case \#4 Changes in Delta

We will do this for a plausible range of deltas 0.10, 0.01, 0.0055 (Atkenson and Burstein 2010). Recall that our baseline is 0.05. 

**Summary** Several things happen from this excercise

1. **Welfare and growth systematically fall as delta decreases.** This is surprising not what my prior would be....

2. The other thing about this is the story about Chi...here as delta gets smaller, Chi is becoming larger! So this is going in the oppisite direction of the GBM case.

**Below are the details and the code**

---

- here is a high delta case ``5.0684    3.2941    0.1496    3.8343``

In [14]:
parameters = parameter_defaults()
parameters = merge(parameters, (,))

d_0 = parameters.d # Here is the 10 percent tariff increase
d_T = 1 + (parameters.d-1)*0.90

#d_autarky = 1 + (parameters.d-1)*2.5 

params_0 = merge(parameters, (d = d_0, )) # parameters to be used at t = 0
params_T = merge(parameters, (d = d_T, )) # parameters to be used at t = T
#params_autarky = merge(parameters, (d = d_autarky, )) # parameters to be used in autarky calc

# initial value for numerical solver on (g, z_hat, Omega)
initial_x = [0.02; 2; .57]

# solve for stationary solution at t = 0
stationary_sol_0 = stationary_algebraic(params_0, initial_x) # solution at t = 0
stationary_sol = stationary_algebraic(params_T, initial_x) # solution at t = T
#stationary_autarky = stationary_algebraic(params_autarky, initial_x) # solution at t = T

lambda_ss = 100*(consumption_equivalent(stationary_sol.U_bar, stationary_sol_0.U_bar, parameters)-1)

print("SS to SS welfare gain: ", lambda_ss,"\n")
print("Change in Trade: ", (1-stationary_sol.λ_ii) - (1-stationary_sol_0.λ_ii),"\n")
print("Growth Rates across SS: ", 100*stationary_sol.g,"   " ,100*stationary_sol_0.g, "\n")

LoadError: syntax: unexpected ","

- low delta case, 0.01: ``5.0684    3.2941    0.0872    7.8101``

In [15]:
parameters = parameter_defaults()
parameters = merge(parameters, (,))

d_0 = parameters.d # Here is the 10 percent tariff increase
d_T = 1 + (parameters.d-1)*0.90

#d_autarky = 1 + (parameters.d-1)*2.5 

params_0 = merge(parameters, (d = d_0, )) # parameters to be used at t = 0
params_T = merge(parameters, (d = d_T, )) # parameters to be used at t = T
#params_autarky = merge(parameters, (d = d_autarky, )) # parameters to be used in autarky calc

# initial value for numerical solver on (g, z_hat, Omega)
initial_x = [0.02; 2; .57]

# solve for stationary solution at t = 0
stationary_sol_0 = stationary_algebraic(params_0, initial_x) # solution at t = 0
stationary_sol = stationary_algebraic(params_T, initial_x) # solution at t = T
#stationary_autarky = stationary_algebraic(params_autarky, initial_x) # solution at t = T

lambda_ss = 100*(consumption_equivalent(stationary_sol.U_bar, stationary_sol_0.U_bar, parameters)-1)

print("SS to SS welfare gain: ", lambda_ss,"\n")
print("Change in Trade: ", (1-stationary_sol.λ_ii) - (1-stationary_sol_0.λ_ii),"\n")
print("Growth Rates across SS: ", 100*stationary_sol.g,"   " ,100*stationary_sol_0.g, "\n")

LoadError: syntax: unexpected ","

 - low delta case, 0.0055: 5.0678    3.2941    0.0841    8.7525
 
 **Note** This is the value that AB(2010) use. They base it on employment, not counts...

In [16]:
parameters = parameter_defaults()
parameters = merge(parameters, ())

d_0 = parameters.d # Here is the 10 percent tariff increase
d_T = 1 + (parameters.d-1)*0.90

#d_autarky = 1 + (parameters.d-1)*2.5 

params_0 = merge(parameters, (d = d_0, )) # parameters to be used at t = 0
params_T = merge(parameters, (d = d_T, )) # parameters to be used at t = T
#params_autarky = merge(parameters, (d = d_autarky, )) # parameters to be used in autarky calc

# initial value for numerical solver on (g, z_hat, Omega)
initial_x = [0.02; 2; .57]

# solve for stationary solution at t = 0
stationary_sol_0 = stationary_algebraic(params_0, initial_x) # solution at t = 0
stationary_sol = stationary_algebraic(params_T, initial_x) # solution at t = T
#stationary_autarky = stationary_algebraic(params_autarky, initial_x) # solution at t = T

lambda_ss = 100*(consumption_equivalent(stationary_sol.U_bar, stationary_sol_0.U_bar, parameters)-1)

print("SS to SS welfare gain: ", lambda_ss,"\n")
print("Change in Trade: ", (1-stationary_sol.λ_ii) - (1-stationary_sol_0.λ_ii),"\n")
print("Growth Rates across SS: ", 100*stationary_sol.g,"   " ,100*stationary_sol_0.g, "\n")

ErrorException: type Array has no field stationary_x0

---

In [17]:
parameters = parameter_defaults()

#3.0065    5.0404    0.0775    6.4481   -0.0188    0.0487    3.1893
parameters = merge(parameters, (,))

d_0 = parameters.d # Here is the 10 percent tariff increase
d_T = 1 + (parameters.d-1)*0.90

#d_autarky = 1 + (parameters.d-1)*2.5 

params_0 = merge(parameters, (d = d_0, )) # parameters to be used at t = 0
params_T = merge(parameters, (d = d_T, )) # parameters to be used at t = T
#params_autarky = merge(parameters, (d = d_autarky, )) # parameters to be used in autarky calc

# initial value for numerical solver on (g, z_hat, Omega)
initial_x = [0.02; 2; .57]

# solve for stationary solution at t = 0
stationary_sol_0 = stationary_algebraic(params_0, initial_x) # solution at t = 0
stationary_sol = stationary_algebraic(params_T, initial_x) # solution at t = T
#stationary_autarky = stationary_algebraic(params_autarky, initial_x) # solution at t = T

lambda_ss = 100*(consumption_equivalent(stationary_sol.U_bar, stationary_sol_0.U_bar, parameters)-1)

print("SS to SS welfare gain: ", lambda_ss,"\n")
print("Change in Trade: ", (1-stationary_sol.λ_ii) - (1-stationary_sol_0.λ_ii),"\n")
print("Growth Rates across SS: ", 100*stationary_sol.g,"   " ,100*stationary_sol_0.g, "\n")

LoadError: syntax: unexpected ","

In [18]:
parameters = parameter_defaults()

parameters = merge(parameters, (,))

d_0 = parameters.d # Here is the 10 percent tariff increase
d_T = 1 + (parameters.d-1)*0.90

#d_autarky = 1 + (parameters.d-1)*2.5 

params_0 = merge(parameters, (d = d_0, )) # parameters to be used at t = 0
params_T = merge(parameters, (d = d_T, )) # parameters to be used at t = T
#params_autarky = merge(parameters, (d = d_autarky, )) # parameters to be used in autarky calc

# initial value for numerical solver on (g, z_hat, Omega)
initial_x = [0.02; 2; .57]

# solve for stationary solution at t = 0
stationary_sol_0 = stationary_algebraic(params_0, initial_x) # solution at t = 0
stationary_sol = stationary_algebraic(params_T, initial_x) # solution at t = T
#stationary_autarky = stationary_algebraic(params_autarky, initial_x) # solution at t = T

lambda_ss = 100*(consumption_equivalent(stationary_sol.U_bar, stationary_sol_0.U_bar, parameters)-1)

print("SS to SS welfare gain: ", lambda_ss,"\n")
print("Change in Trade: ", (1-stationary_sol.λ_ii) - (1-stationary_sol_0.λ_ii),"\n")
print("Growth Rates across SS: ", 100*stationary_sol.g,"   " ,100*stationary_sol_0.g, "\n")

LoadError: syntax: unexpected ","

In [19]:
parameters = parameter_defaults()
#3.0426    5.0018    0.0732    5.9577   -0.0189    0.0484    3.1725
parameters = merge(parameters, (ρ = 0.0215, d = 3.0426 , θ = 5.0018  ,  κ = 0.0732, χ = 1/5.9577, υ =  0.0484 , μ = -0.0189, σ = 3.1725, δ = 0.02,))

d_0 = parameters.d # Here is the 10 percent tariff increase
d_T = 1 + (parameters.d-1)*0.90

#d_autarky = 1 + (parameters.d-1)*2.5 

params_0 = merge(parameters, (d = d_0, )) # parameters to be used at t = 0
params_T = merge(parameters, (d = d_T, )) # parameters to be used at t = T
#params_autarky = merge(parameters, (d = d_autarky, )) # parameters to be used in autarky calc

# initial value for numerical solver on (g, z_hat, Omega)
initial_x = [0.02; 2; .57]

# solve for stationary solution at t = 0
stationary_sol_0 = stationary_algebraic(params_0, initial_x) # solution at t = 0
stationary_sol = stationary_algebraic(params_T, initial_x) # solution at t = T
#stationary_autarky = stationary_algebraic(params_autarky, initial_x) # solution at t = T

lambda_ss = 100*(consumption_equivalent(stationary_sol.U_bar, stationary_sol_0.U_bar, parameters)-1)

acr_gains = 100*(((stationary_sol.λ_ii/stationary_sol_0.λ_ii))^(-1 / parameters.θ) - 1)

print("SS to SS welfare gain: ", lambda_ss,"\n")
print("Change in Trade: ", (1-stationary_sol.λ_ii) - (1-stationary_sol_0.λ_ii),"\n")
print("Growth Rates across SS: ", 100*stationary_sol.g,"   " ,100*stationary_sol_0.g, "\n")
print("ACR Gains: ", acr_gains, "\n")

ErrorException: type Array has no field stationary_x0

In [20]:
acr_gains

UndefVarError: UndefVarError: acr_gains not defined

In [21]:
d_T 

2.8383400000000005